In [ ]:
##############import#################
import h5py
import pickle
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import random

import tensorflow as tf
tf.random.set_seed(33)

from tensorflow.keras import backend as K
# from tensorflow import keras
from tensorflow.keras import layers,models,optimizers,callbacks,constraints
from tensorflow.keras.constraints import Constraint
from tensorflow.keras.layers import Input, Dense, Conv2D,multiply,Lambda,Add,Concatenate, Multiply,Conv2DTranspose,Layer, Reshape, ZeroPadding2D,Flatten, MaxPooling2D, UpSampling2D, BatchNormalization,LeakyReLU
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,Callback,LearningRateScheduler,ModelCheckpoint
# set_session(tf.Session(config=config))
from sklearn.utils import shuffle
import math
import os
from tensorflow.keras import regularizers

gpus = tf.config.list_physical_devices('GPU')

jobid=int(0)
param={i:[] for i in range(11)}
param[0]=[2,1000,5,1000,500,15,'ppp']###################

param[1]=[3,1,1000,1,10,20,'sr']
param[2]=[2,1000,10,1,10,20,'sr']###
param[3]=[2,0.001,1,1,5,20,'sr']###
param[4]=[2,0.001,5,1,5,30,'sr']
param[5]=[2,0.001,10,1,10,20,'sr']####
param[6]=[3,100,5,1,10,20,'sr']###
#############hyperparameter#################
background=2
udim=param[jobid][0]###
sdim=5
latent_dim=background+udim+sdim
bac=2
alpha=param[jobid][1]###
beta=param[jobid][2]####
gamma=param[jobid][3]####
theta=param[jobid][4]####
ks=param[jobid][5]####
flag=param[jobid][6]####
batch_size=10

hdf5_file = 'data_nv30364657.hdf5' #Data available at: https://drive.google.com/file/d/1efgnHQ9ZXahn3-Z-boX9awmdq04XFKmZ/view?usp=sharing

a = h5py.File(hdf5_file, 'r')

name=list(a.keys())
i1=a[name[1]]
trail=list(i1.keys())
# i12=i1[trail[1]]
# # i12
image_name=['images_30', 'images_36', 'images_46', 'images_57']
label_name=['labels_30', 'labels_36', 'labels_46', 'labels_57']
# print(name)
va={i:[] for i in range (4)}

for i in range (4):
    imn=image_name[i]
    lbn=image_name[i]
    for j in trail:
#         print(imn)
        data=np.array(a[imn][j])
        v=np.var(data,axis=0)
        va[i].append(np.mean(v))

so={i:[] for i in range (4)}
vv={i:[] for i in range (4)}
for i in range (4):
    so[i]=np.argsort(np.array(va[i])*-1)
    vv[i]=np.sort(np.array(va[i])*-1)


z=0
idx={i:[] for i in range (388*2*189)}
for i in range(4):
    seq=so[i]
    for j in seq[:194]:
        for k in range (189):
            n=trail[j]
            idx[z].append(i)
            idx[z].append(n)
            idx[z].append(k)
            z+=1

idx2=shuffle(idx)


idx={i:[] for i in range (388*2*189)}
z=0
for k in range (189):
    for i in range(4):
        seq=so[i]
        for j in seq[int(len(trail)/2):]:
            n=trail[j]
    #         print(n)

            idx[z].append(i)
            idx[z].append(n)
            idx[z].append(k)
            z+=1
# len(idx)
idx3=shuffle(idx)
index=idx2+idx3

def data_generator(idx, hdf5file,batch_size,if_train = True):
    i = 0
#     j=0 
    while True:
        X = []
        Y=[]
        for b in range(batch_size):
            if i == len(idx):
                i = 0
            
            alll=idx[i]
            if alll[0]==0:
                imn='images_30'
                lbn='labels_30'

            elif alll[0]==1:
                imn='images_36'
                lbn='labels_36'
            elif alll[0]==2:
                imn='images_46'
                lbn='labels_46'
            else:
                imn='images_57'
                lbn='labels_57'
                
            try:
                x = (np.array(hdf5file[imn][alll[1]][alll[2]])/255).T  # read dataset on the fly
                x=np.rollaxis(x,1,0)

                y = np.array(hdf5file[lbn][alll[1]][alll[2]])  # read dataset on the fly


                if np.sum(x)==0 or np.sum(x) == np.inf or np.sum(x) == -np.inf or np.sum(y)==0 or np.sum(y)==np.inf or np.sum(y)==-np.inf:
                    x = (np.array(hdf5file[imn]['trial_0030'][60])/255).T  # read dataset on the fly
                    x=np.rollaxis(x,1,0)
                    y = np.array(hdf5file[lbn]['trial_0030'][60])  # read dataset on the fly
            except:
                x = (np.array(hdf5file[imn]['trial_0030'][60])/255).T  # read dataset on the fly
                x=np.rollaxis(x,1,0)
                y = np.array(hdf5file[lbn]['trial_0030'][60])  # read dataset on the fly
            

                
            X.append(x)
            Y.append(y)

            i += 1
#             j += 1

        X = np.asarray(X)
        Y = np.asarray(Y)
        yield [X,Y],[X,Y]
        
# idx=np.arange(len(image))
###############################################
##################define classes for loss and regularizor######
def Distance(x, y):
    d = tf.reduce_sum(tf.square(x - y), 1)
    dx = tf.maximum(d, 1e-9)
    dist = tf.sqrt(dx)
    return tf.reduce_mean(dist)


def swiss_roll(size, noise=1.5):
    u = K.random_uniform(shape=(size[0], 1),dtype="float32")
    u2 = K.random_uniform(shape=(size[0], 1),dtype="float32")*10

    t = 1.5 * np.pi * (1 + 1.5* u)
    x = t * K.cos(t)#+11*K.ones_like(u2)
    y =  u2 #+ 11*K.ones_like(u2)
    z = t * K.sin(t) #+ 11*K.ones_like(u2)

    X = K.concatenate([x,z], axis=-1)

    return X

def squre_roll(size, noise=1.5):
    u =K.random_uniform(shape=(size[0], 1),dtype="float32")
    u2 = K.random_uniform(shape=(size[0], 1),dtype="float32")*10
#     print(u)
    t = 1.5 * np.pi * (1 + 100 * u)
    x = t * K.cos(t)#+11*K.ones_like(u2)
    y =  u2# + 11*K.ones_like(u2)
    z = t * K.sin(t) #+ 11*K.ones_like(u2)

    X = K.concatenate([x,y,z], axis=-1)

    return X
def spherical(size, ndim=3):
    vec = K.random_normal(shape=(size[0], ndim),dtype="float32")
#     vec /= K.linalg.norm(vec, axis=0)
    return vec

def clusterdis(size, noise=1.5):
    
    a=int(size[0]/4)
    b=a
    c=a
    d=size[0]-a-b-c
    u = K.random_uniform(shape=(a, 3),dtype="float32")
    u2 = K.random_uniform(shape=(b, 3),dtype="float32")*2
    u3 = K.random_uniform(shape=(c, 3),dtype="float32")*4
    u4 = K.random_uniform(shape=(d, 3),dtype="float32")*8
#     sess = tf.Session()
#     with sess.as_default():
#         t=K.eval(size[0])
    X = K.concatenate([u,u2,u3,u4], axis=0)

#     X,_=make_blobs(n_samples=t, centers=4, n_features=3,random_state=42)
    return X

def GMM(size,noise=1.5):
    
    N,D = size[0], 3 # number of points and dimenstinality

    means = np.array([[0.5, 0.0, 0.0],
                      [0.0, 0.5, 0.8],
                      [-0.5, -0.5, -0.5],
                      [-0.8, 0.3, 0.4]])
#     means= K.constant(means)

    covs = np.array([np.diag([0.01, 0.01, 0.01]),
                     np.diag([0.01, 0.01, 0.01]),
                     np.diag([0.01, 0.01, 0.01]),
                     np.diag([0.01, 0.01, 0.01])])
#     covs=K.constant(covs)

    n_gaussians = means.shape[0]

    points = []
    if N % 4 == 0:
        L=int(N/4)
        
        
    for i in range(len(means)):
        x = np.random.multivariate_normal(means[i], covs[i],L)
        points.append(x)

    points=K.constant(np.concatenate(points))
    
    return points

def get_kernel(X, Z,ksize):
#     print(X.shape,Z.shape)
    G = K.sum((K.expand_dims(X, axis=1) - Z)**2, axis=-1)  # Gram matrix
    G = K.exp(-G/(ksize)) / (math.sqrt(2*np.pi*ksize)*K.ones_like(-G/(ksize)))
    return G

class DiagonalWeight(Constraint):
    """Constrains the weights to be diagonal.
    """
    def __init__(self, N):
        self.m = K.eye(N)
        
    def __call__(self, w):
#         N = K.int_shape(w)[-1]
#         m = K.eye(N)
        w = w*self.m
        return w
    
class KLDivergenceLayer(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_var = inputs

        kl_batch =  -0.5 * K.sum(1 + log_var -
                                K.square(mu) - K.exp(log_var), axis=-1)

        self.add_loss(tf.reduce_mean(kl_batch), inputs=inputs)

        return inputs,tf.reduce_mean(kl_batch)

class ITLRegularizer(Layer):

    """ Identity transform layer that adds cs divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(ITLRegularizer, self).__init__(*args, **kwargs)


    
    def call(self, inputs,ks,flag,theta):

        X = inputs
#         print(X)
        if flag=='sp':
            Z=spherical(K.shape(X))
        elif flag == 'cb':
            Z=clusterdis(K.shape(X))
        elif flag == 'gmm':
        
            Z=GMM(K.shape(X))
        elif flag == 'sq':
            Z=squre_roll(K.shape(X))
        else:
            Z=swiss_roll(K.shape(X))

    

#         Z=Z.astype("float32")
        ksize=ks
        Gxx = get_kernel(X, X,ksize)
        Gzz = get_kernel(Z, Z,ksize)
        Gxz = get_kernel(X, Z,ksize)


        r=K.log(K.sqrt(K.mean(Gxx)*K.mean(Gzz)+1e-5) /(K.mean(Gxz)+1e-5))

        self.add_loss(r*theta, inputs=inputs)

        return inputs,Z,r*theta
    
class OrthLayer(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(OrthLayer, self).__init__(*args, **kwargs)

    def call(self, inputs,gamma):

        A,B = inputs
        U=K.concatenate((A,B),axis=1)
        U = K.l2_normalize(U, axis=1)
        batch = -K.dot(K.transpose(U),U) 
        I =tf.eye(K.shape(U)[1])#tf.Variable(lambda:K.eye(K.shape(U)[1])) #Lambda(lambda t: K.eye(t))()
        batch=(batch+I)**2*gamma
        self.add_loss(tf.reduce_mean(batch), inputs=inputs)

        return inputs,tf.reduce_mean(batch)

    
    
class MSE_SUP(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(MSE_SUP, self).__init__(*args, **kwargs)
    def call(self,inputs, alpha):
        D,A=inputs
        L=tf.keras.losses.mse(D,A)
        L=tf.reduce_mean(L)
        
        self.add_loss(L*alpha, inputs=inputs)
        
        return inputs,L*alpha
        
    
class MSE_UNSUP(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(MSE_UNSUP, self).__init__(*args, **kwargs)
    def call(self,inputs):
        D,A=inputs

        L=tf.keras.losses.mse(D,A)
        L=tf.reduce_mean(L)
        self.add_loss(L*128*128*2, inputs=inputs)
        
        return inputs,L*128*128*2
    
    
LN2PI=np.log(2 * 3.1415926)



class De_KL(Layer):
    
    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(De_KL, self).__init__(*args, **kwargs)
        
    def _gaussian_log_density_unsummed(self,z, mu, logvar):
        """First step of Gaussian log-density computation, without summing over dimensions.
        Assumes a diagonal noise covariance matrix.
        """
        diff_sq = (z - mu) ** 2
        inv_var = K.exp(-logvar)
        return - 0.5 * (inv_var * diff_sq + logvar + LN2PI)


    def _gaussian_log_density_unsummed_std_normal(self,z):
        """First step of Gaussian log-density computation, without summing over dimensions.
        Assumes a diagonal noise covariance matrix.
        """
        diff_sq = z ** 2
        return - 0.5 * (diff_sq + LN2PI)

    
    def _logsumexp(self,x, axis=None, keepdims=False):

        x = tf.convert_to_tensor(x)
        x_max = tf.math.reduce_max(x, axis=axis, keepdims=False)
        temp=tf.math.reduce_sum(K.exp(x - x_max)+1, axis=axis,keepdims=True)
        ret = K.log(tf.math.reduce_max(temp+1,tf.zeros_like(temp+1))) + x_max
#         np.log(np.max(x, 1e-9))
        if not keepdims:
            ret = tf.math.reduce_sum(ret, axis=axis)
#         print(K.sum(x,axis=1).numpy())
        return ret#K.sum(x,axis=1)

    
    def call(self,inputs,beta):
        z, mu, logvar=inputs
        log_qz_prob = self._gaussian_log_density_unsummed(z[:, None], mu[None, :], logvar[None, :])
        M =tf.nn.relu(K.sum(log_qz_prob, axis=2, keepdims=False))#,axis=1,keepdims=False)
        c =tf.nn.relu(log_qz_prob)#,axis=1,keepdims=False)
        t1=K.sum(K.exp(-M)+K.exp(K.sum(log_qz_prob, axis=2, keepdims=False)-M),axis=1,keepdims=False)
        log_qz= K.log(t1+1e-5)
        #tf.reduce_logsumexp(K.sum(log_qz_prob, axis=2, keepdims=False),axis=1,keepdims=False)
       
        
        #self._logsumexp(K.sum(log_qz_prob, axis=2, keepdims=False),axis=1,keepdims=False)
        
        
#         print(log_qz.shape)
        
        log_qz_ = tf.linalg.diag(M) # sum over gaussian dims
        
#         print(log_qz_.shape)
        t2=K.sum(K.exp(-c)+K.exp(log_qz_prob-c),axis=1,keepdims=False)
        log_qz_product = K.sum(#log_qz_prob,
#             K.sum(K.log(K.exp(-c)+K.exp(log_qz_prob-c)),axis=1,keepdims=False),
#             self._logsumexp(log_qz_prob, axis=1,keepdims=False),
            K.log(t2+1e-5),
#             tf.reduce_logsumexp(log_qz_prob, axis=1,keepdims=False),  # logsumexp over batch
            axis=1,  # sum over gaussian dims
            keepdims=False)
        
        log_pz_prob = self._gaussian_log_density_unsummed_std_normal(z)
        log_pz_product = K.sum(log_pz_prob, axis=1, keepdims=False)  # sum over gaussian dims
#         print(log_qz_.shape , log_qz.shape)
        idx_code_mi = tf.experimental.numpy.nanmean(log_qz_ - log_qz)
        total_corr = tf.experimental.numpy.nanmean(log_qz - log_qz_product)
        dim_wise_kl = tf.experimental.numpy.nanmean(log_qz_product - log_pz_product)
        idx_code_mi = tf.reduce_mean(idx_code_mi )
        total_corr = tf.reduce_mean(total_corr )
        dim_wise_kl = tf.reduce_mean(dim_wise_kl)
        self.add_loss((idx_code_mi+total_corr *beta+dim_wise_kl+1e-5), inputs=inputs)
        
        return inputs,[idx_code_mi,total_corr *beta,dim_wise_kl]
###############################################
input0 = Input(shape=(128,128,2), name="mice1")
# encoder=Sequential()
inputlabel=Input(shape=(sdim), name="micelabel1")

encoder=LeakyReLU(alpha=0.05)(Conv2D(32, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")(input0))
encoder=BatchNormalization()(encoder)
# print(encoder.shape)
encoder=LeakyReLU(alpha=0.05)(Conv2D(64, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")(encoder))
encoder=BatchNormalization()(encoder)
# print(encoder.shape)
encoder=LeakyReLU(alpha=0.05)(Conv2D(128, (5,5), strides=(2,2),padding='same', kernel_initializer="he_normal")(encoder))
encoder=BatchNormalization()(encoder)
# print(encoder.shape)
encoder=LeakyReLU(alpha=0.05)(Conv2D(256, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")(encoder))
encoder=BatchNormalization()(encoder)
# print(encoder.shape)
encoder=LeakyReLU(alpha=0.05)(Conv2D(512, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal")(encoder))
encoder=BatchNormalization()(encoder)
# print(encoder.shape)
hidden=Flatten()(encoder)
# hidden.shape

z_mu = Dense(latent_dim,kernel_regularizer='l1_l2')(hidden)
z_log_var = Dense(sdim+udim,kernel_regularizer='l1_l2')(hidden)
#

from scipy.stats import ortho_group
m = ortho_group.rvs(dim=latent_dim).astype('float32')

from scipy.linalg import qr

n = sdim+udim+bac
H = np.random.randn(n, n)
Q, _ = qr(H)

initializer = tf.keras.initializers.Orthogonal()
initializera=tf.keras.initializers.Orthogonal(gain=1.0, seed=42)
initializerb=tf.keras.initializers.Orthogonal(gain=2.0, seed=30)


class FixWeights_sup(tf.keras.constraints.Constraint):

    def __call__(self, w):
        tf.keras.backend.set_value(w, Q[:sdim,:sdim])
        return w
class FixWeights_unsup(tf.keras.constraints.Constraint):

    def __call__(self, w):
        tf.keras.backend.set_value(w, Q[sdim:sdim+udim,sdim:sdim+udim])
        return w
    
class FixWeights_bac(tf.keras.constraints.Constraint):

    def __call__(self, w):
        tf.keras.backend.set_value(w, Q[sdim+udim:sdim+udim+bac-1,sdim+udim:sdim+udim+bac-1])
        return w

C1 = Dense(bac, use_bias=False,kernel_regularizer='l1_l2')#(z_mu)
A1 = Dense(sdim,use_bias=False,kernel_initializer=initializera,kernel_regularizer='l1_l2')#(z_mu)
B1 = Dense(udim, use_bias=False, kernel_initializer=initializerb,kernel_regularizer='l1_l2')#(z_mu)


z_log_var_A=z_log_var[:,:sdim]
z_log_var_B=z_log_var[:,sdim:]



A1.trainable = False
B1.trainable = False

A2 = A1(z_mu)
B = B1(z_mu)
C = C1(z_mu)

C,dis,loss2= ITLRegularizer()(C,ks,flag,theta)

D1=Dense(sdim,use_bias=True,kernel_constraint=DiagonalWeight(sdim), kernel_initializer="he_normal")(A2)
[D,A3],loss3=MSE_SUP()([D1,inputlabel],alpha)

[A, z_log_var_A2],loss4 = KLDivergenceLayer()([A2, z_log_var[:,:sdim]])
z_sigma_A = Lambda(lambda t: K.exp(.5*t))(z_log_var_A2)

eps_A = K.random_normal(shape=(K.shape(input0)[0],sdim))
z_eps_A2 = Multiply()([z_sigma_A, eps_A])#Multiply()([z_sigma_A, eps_A])
z_A = Add()([A, z_eps_A2])

z_sigma_B = Lambda(lambda t: K.exp(.5*t))(z_log_var[:,sdim:])
eps_B = K.random_normal(shape=(K.shape(input0)[0],udim))
z_eps_B = Multiply()([z_sigma_B, eps_B])
z_B = Add()([B, z_eps_B])

[z_B1, B1, z_log_var_B1],[loss5,loss6,loss7]= De_KL()([z_B, B, z_log_var_B],beta)


alla=Concatenate()([z_A,z_B1,C])


# print(zall.shape)
decoder = Sequential()
decoder.add(Dense(hidden.shape[1], kernel_initializer="he_normal"))
decoder.add(Reshape((4, 4, 512)))
decoder.add(Conv2DTranspose(256, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal"))
decoder.add(LeakyReLU(alpha=0.05))
decoder.add(BatchNormalization())

decoder.add(Conv2DTranspose(128, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal"))
decoder.add(LeakyReLU(alpha=0.05))
decoder.add(BatchNormalization())

decoder.add(Conv2DTranspose(64, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal"))
decoder.add(LeakyReLU(alpha=0.05))
decoder.add(BatchNormalization())

decoder.add(Conv2DTranspose(32, (5,5), strides= (2,2),padding='same', kernel_initializer="he_normal"))
decoder.add(LeakyReLU(alpha=0.05))
decoder.add(BatchNormalization())

decoder.add(Conv2DTranspose(2, (5,5), strides= (2,2),activation='sigmoid',padding='same', kernel_initializer="he_normal"))

encmodel = Model(inputs=[input0,inputlabel], outputs=alla)

out=decoder(alla)
[out,_],loss8=MSE_UNSUP()([out,input0])



allmodel= Model(inputs=[input0,inputlabel], outputs=[out,D])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001,clipvalue=1.0)

allmodel.add_metric(loss2, "csd_loss")
allmodel.add_metric(loss3, "sup_loss")
allmodel.add_metric(loss4, "kl_loss")
allmodel.add_metric(loss5, "mi_loss")
allmodel.add_metric(loss6, "total_corr")
allmodel.add_metric(loss7, "dim_wise_kl")
allmodel.add_metric(loss8, "UNSUP_loss")

allmodel.compile( optimizer=optimizer)


callbacks=[LearningRateScheduler(lambda epoch: 0.001 * 0.85 ** (epoch // 5))]
term=tf.keras.callbacks.TerminateOnNaN()



history1 =allmodel.fit( data_generator(index, a,256,True),validation_data=data_generator(index[-10000:],a,100,False),
                       batch_size=256, epochs=50,validation_steps=np.ceil(10000/100-1),
                       verbose=1, steps_per_epoch=np.ceil((len(index))/256-1),
                       callbacks=[term,callbacks])

allmodel.save_weights('model.h5')

# allmodel.load_weights('csdmodel_4_udim=2_alpha=1000_beta=5_gamma=1000_theta=500_ks=15_flag=sror3.h5')###!!!!!!!


            